In [1]:
# Hellwarth 1999 PRB - Part IV; T-dep of the Feynman variation parameter
# A Friday afternoon of hacking to try and implement the T-dep electron-phonon coupling from the above PRB
# Which was unusually succesful! And more or less reproduced Table III

In [2]:
# Just in case anyone is following this from the far future; we are using:
versioninfo()

Julia Version 0.5.0
Commit 3c9d753 (2016-09-19 18:14 UTC)
Platform Info:
  System: Darwin (x86_64-apple-darwin13.4.0)
  CPU: Intel(R) Core(TM) i7-6700K CPU @ 4.00GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Haswell)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.7.1 (ORCJIT, broadwell)


In [3]:
# one-dimensional numerical integration in Julia using adaptive Gauss-Kronrod quadrature
using QuadGK

# Equation numbers follow above Hellwarth 1999 PRB
# 62b
A(v,w,β)=3/β*( log(v/w) - 1/2*log(2*π*β) - log(sinh(v*β/2)/sinh(w*β/2)))

# 62d
Y(x,v,β)=1/(1-exp(-v*β))*(1+exp(-v*β)-exp(-v*x)-exp(v*(x-β)))
# 62c integrand
f(x,v,w,β)=(exp(β-x)+exp(x))/(w^2*x*(1-x/β)+Y(x,v,β)*(v^2-w^2)/v)^(1/2)
# 62c
B(v,w,β,α) = α*v/(sqrt(π)*(exp(β)-1)) * quadgk(x->f(x,v,w,β),0,β/2)[1]
#62e
C(v,w,β)=3/4*(v^2-w^2)/v * (coth(v*β/2)-2/(v*β))

F(v,w,β,α)=-(A(v,w,β)+B(v,w,β,α)+C(v,w,β)) #(62a)

# Can now evaluate, e.g.
# F(v,w,β,α)=F(7.2,6.5,1.0,1.0)
# BUT - this is just the objective function! Not the optimised parameters.
# Also there's a scary numeric integration (quadgk) buried within...

F (generic function with 1 method)

In [4]:
# Print out F(alpha,beta) for a specific v,w; as a test
@printf("\t\t")
for α in 1:5
    @printf("α=%d\t\t",α)
end
@printf("\n")

for β in 1:0.25:3.0
    v=w=4
    print("β: $β  \t||")
    for α in 1:5
        @printf("%f\t",F(v,w,β,α))
    end
    println()
end

		α=1		α=2		α=3		α=4		α=5		
β: 1.0  	||0.948149	-0.860517	-2.669184	-4.477850	-6.286517	
β: 1.25  	||0.837826	-0.797573	-2.432973	-4.068372	-5.703771	
β: 1.5  	||0.731167	-0.781009	-2.293184	-3.805360	-5.317535	
β: 1.75  	||0.634483	-0.786028	-2.206538	-3.627049	-5.047560	
β: 2.0  	||0.548050	-0.802169	-2.152387	-3.502605	-4.852824	
β: 2.25  	||0.470735	-0.824402	-2.119538	-3.414675	-4.709811	
β: 2.5  	||0.401226	-0.850049	-2.101324	-3.352599	-4.603874	
β: 2.75  	||0.338349	-0.877564	-2.093476	-3.309388	-4.525300	
β: 3.0  	||0.281128	-0.905989	-2.093106	-3.280223	-4.467340	


In [5]:
# OK - very primitive!
# But these are 1D traces along the solution for Alpha=Beta=1 in Helwarth PRB TABLE III,
# this was used to correct a transcription error in the above typed-in equations
# It was also good to see what F(v,w) looked like as a function of v and w near an optimal solution

v=7.20
w=6.5
α=1.0
β=1.0

for v=6:0.1:8
    @printf("%f %f\n",v,F(v,w,β,α))
end

@printf("\n")
v=7.20
for w=6:0.1:7
    @printf("%f %f\n",w,F(v,w,β,α))
end

6.000000 0.991013
6.100000 0.980559
6.200000 0.971051
6.300000 0.962484
6.400000 0.954852
6.500000 0.948149
6.600000 0.942368
6.700000 0.937501
6.800000 0.933540
6.900000 0.930475
7.000000 0.928297
7.100000 0.926997
7.200000 0.926565
7.300000 0.926990
7.400000 0.928261
7.500000 0.930368
7.600000 0.933300
7.700000 0.937045
7.800000 0.941592
7.900000 0.946930
8.000000 0.953047

6.000000 0.936794
6.100000 0.933154
6.200000 0.930295
6.300000 0.928232
6.400000 0.926983
6.500000 0.926565
6.600000 0.926993
6.700000 0.928281
6.800000 0.930446
6.900000 0.933502
7.000000 0.937462


In [6]:
# Angle for the ringside seats, when the fall, don't blame me, Bring on the Major Leagues
using Optim
# Julia package stuffed full of magic, does auto-differentation & etc. etc.

In [7]:
Fopt(x) = F(x[1],x[2],1,1)

Fopt([7.2,6.5])
# OK! It looks like I can bury the alpha, beta parameters (which we don't optimise), by wrapping our function in a function definition.

0.9265650282717047

In [8]:
initial=[7.2,6.5]

optimize(Fopt,  initial, LBFGS())

Results of Optimization Algorithm
 * Algorithm: L-BFGS
 * Starting Point: [7.2,6.5]
 * Minimizer: [7.200120820337425,6.4998864063978345]
 * Minimum: 9.265650e-01
 * Iterations: 5
 * Convergence: true
   * |x - x'| < 1.0e-32: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: true
   * |g(x)| < 1.0e-08: false
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 20
 * Gradient Calls: 20

In [9]:
optimize(Fopt, initial, BFGS(), Optim.Options(autodiff=true))

Results of Optimization Algorithm
 * Algorithm: BFGS
 * Starting Point: [7.2,6.5]
 * Minimizer: [7.2029649846871795,6.502749233940956]
 * Minimum: 9.265650e-01
 * Iterations: 3
 * Convergence: true
   * |x - x'| < 1.0e-32: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: true
   * |g(x)| < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 19
 * Gradient Calls: 19

In [10]:
optimize(Fopt, initial, BFGS(), Optim.Options(autodiff=true))

Results of Optimization Algorithm
 * Algorithm: BFGS
 * Starting Point: [7.2,6.5]
 * Minimizer: [7.2029649846871795,6.502749233940956]
 * Minimum: 9.265650e-01
 * Iterations: 3
 * Convergence: true
   * |x - x'| < 1.0e-32: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: true
   * |g(x)| < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 19
 * Gradient Calls: 19

In [11]:
# Right! The above looks like this might actually just work...

@printf("\t\t")
for α in 1:5
    @printf("α=%d\t\t",α)
end
@printf("\n")

for β in 1:0.25:3.0
    print("β: $β  \t||")
    for α in 1:5
        myf(x) = F(x[1],x[2],β,α)
        solution=Optim.minimizer(optimize(myf, initial,ConjugateGradient(), Optim.Options(autodiff=true)))
        
        #print(solution,"\t")
        @printf("%.2f %.2f\t",solution[1],solution[2])
    end
    println()
end

		α=1		α=2		α=3		α=4		α=5		
β: 1.0  	||7.20 6.50	7.69 6.20	8.26 5.87	8.95 5.51	9.78 5.13	
β: 1.25  	||5.94 5.31	6.38 5.03	6.91 4.73	7.57 4.41	8.39 4.05	
β: 1.5  	||5.11 4.54	5.52 4.29	6.02 4.00	6.66 3.70	7.48 3.37	
β: 1.75  	||4.54 4.01	4.91 3.76	5.40 3.51	6.02 3.22	6.83 2.92	
β: 2.0  	||4.12 3.63	4.48 3.40	4.94 3.16	

LoadError: DomainError:

In [12]:
# After a bit of fiddling, I figured out how to add bounds, to stop that 'DomainError', 
# which occurs where the you are evaluating log(-ve Real), i.e. w<0.0 or v<0.0

initial=[7.2,6.5]

lower=[0.0,0.0]
upper=[10.0,10.0]

@printf("\t\t")
for α in 1:5
    @printf("α=%d\t\t",α)
end
@printf("\n")

for β in 1:0.25:3.0
    print("β: $β  \t||")
    for α in 1:5
        myf(x) = F(x[1],x[2],β,α)
        solution=optimize(DifferentiableFunction(myf), initial, lower, upper, Fminbox(); optimizer = ConjugateGradient, optimizer_o=Optim.Options(autodiff=true))
        minimum=Optim.minimizer(solution)
                
        #print(solution,"\t")
        @printf("%.2f %.2f\t",minimum[1],minimum[2])
    end
    println()
end

		α=1		α=2		α=3		α=4		α=5		
β: 1.0  	||7.

20 6.50	7.60 6.11	8.28 5.89	8.95 5.52	9.77 5.12	
β: 1.25  	||5.88 5.26	6.40 5.05	6.92 4.74	7.57 4.41	8.39 4.06	
β: 1.5  	||5.08 4.51	5.52 4.28	6.01 4.00	6.65 3.70	7.48 3.38	
β: 1.75  	||4.58 4.05	4.93 3.78	5.39 3.50	6.02 3.23	6.84 2.92	
β: 2.0  	||4.14 3.65	4.49 3.41	4.94 3.16	5.54 2.88	6.37 2.60	
β: 2.25  	||3.80 3.33	4.17 3.16	4.58 2.89	5.18 2.64	6.00 2.37	
β: 2.5  	||3.58 3.14	3.90 2.93	4.32 2.70	4.90 2.45	5.71 2.20	
β: 2.75  	||3.39 2.97	3.69 2.77	4.10 2.55	4.67 2.31	5.48 2.07	
β: 3.0  	||3.25 2.85	3.53 2.65	3.93 2.43	4.48 2.21	5.28 1.97	


In [13]:
# So that looks really good! I was super stoked to see how close these values are to TABLE III in Hellwarth
# However, the solutions all start on (7.20,6.50) so that top-left data point is cheating, whereas the 
# others have some disagreement / noise associated with them
# I was wondering whether it might be a function of the optimiser, so thought I'd try them all

initial=[7.1,6.5]
# Main use of these bounds is stopping v or w going negative, at which you get a NaN error as you are evaluating log(-ve Real)
lower=[1.0,1.0]
upper=[10.0,10.0]

for optimizer in [BFGS, LBFGS, ConjugateGradient] # Newton, GradientDescent, NelderMead - steps outside box & log(-ve)->NaN error
    @printf("\n\t\t##### NOW TRIALING: %s #####\n\n",optimizer)

    @printf("\t\t")
    for α in 1:5
        @printf("α=%d\t\t",α)
    end
    @printf("\n")

    for β in 1:0.25:3.0
        print("β: $β  \t||")
        for α in 1:5
            myf(x) = F(x[1],x[2],β,α)
            res=optimize(DifferentiableFunction(myf), initial, lower, upper, Fminbox(); optimizer = optimizer, optimizer_o=Optim.Options(autodiff=true))
            minimum=Optim.minimizer(res)
            #show(Optim.converged(res)) # All came out as 'true'
                
            #print(solution,"\t")
            @printf("%.2f %.2f\t",minimum[1],minimum[2])
        end
        println()
    end
end


		##### NOW TRIALING: Optim.BFGS{L<:Function,H<:Function} #####

		α=1		α=2		α=3		α=4		α=5		
β: 1.0  	||7.15 6.45	7.69 6.20	8.17 5.78	8.94 5.50	9.77 5.12	
β: 1.25  	||5.93 5.30	6.38 5.03	6.90 4.73	7.58 4.42	8.38 4.04	
β: 1.5  	||5.12 4.55	5.55 4.32	6.02 4.00	6.67 3.72	7.48 3.37	
β: 1.75  	||4.55 4.02	4.94 3.79	5.40 3.51	6.02 3.22	6.84 2.92	
β: 2.0  	||4.16 3.66	4.48 3.40	4.95 3.17	5.54 2.88	6.36 2.60	
β: 2.25  	||3.83 3.37	4.16 3.14	4.60 2.90	5.17 2.63	6.00 2.37	
β: 2.5  	||3.60 3.16	3.89 2.93	4.32 2.70	4.90 2.45	5.71 2.20	
β: 2.75  	||3.38 2.96	3.68 2.75	4.11 2.56	4.67 2.31	5.48 2.07	
β: 3.0  	||3.26 2.86	3.53 2.64	3.93 2.43	4.48 2.21	5.29 1.97	

		##### NOW TRIALING: Optim.LBFGS{T,L<:Function,Tprep<:Union{Function,Void}} #####

		α=1		α=2		α=3		α=4		α=5		
β: 1.0  	||7.21 6.51	7.71 6.23	8.21 5.82	8.95 5.52	9.79 5.13	
β: 1.25  	||5.95 5.32	6.36 5.01	6.90 4.72	7.55 4.39	8.39 4.05	
β: 1.5  	||5.13 4.56	5.53 4.29	6.02 4.00	6.64 3.69	7.48 3.38	
β: 1.75  	||4.53 4.01	4.93 3.79	5.39 3.50	6.

4.93 3.15	5.54 2.89	6.36 2.61	
β: 2.25  	||3.83 3.36	4.14 3.13	4.58 2.89	5.18 2.64	6.00 2.37	
β: 2.5  	||3.57 3.13	3.90 2.93	4.32 2.70	4.90 2.45	5.71 2.20	
β: 2.75  	||3.39 2.97	3.69 2.77	4.10 2.55	4.67 2.31	5.48 2.07	
β: 3.0  	||3.22 2.82	3.54 2.65	3.93 2.44	4.49 2.21	5.29 1.97	

		##### NOW TRIALING: Optim.ConjugateGradient{T,Tprep<:Union{Function,Void},L<:Function} #####

		α=1		α=2		α=3		α=4		α=5		
β: 1.0  	||7.19 6.49	7.67 6.18	8.25 5.86	8.96 5.53	9.79 5.13	
β: 1.25  	||5.91 5.28	6.35 5.01	6.89 4.72	7.56 4.40	8.39 4.05	
β: 1.5  	||5.11 4.54	5.51 4.28	6.02 4.00	6.66 3.70	7.48 3.37	
β: 1.75  	||4.53 4.00	4.93 3.78	5.39 3.51	6.01 3.22	6.84 2.92	
β: 2.0  	||4.11 3.61	4.49 3.41	4.94 3.15	5.54 2.88	6.36 2.60	
β: 2.25  	||3.81 3.35	4.16 3.14	4.60 2.90	5.18 2.64	6.00 2.37	
β: 2.5  	||3.58 3.14	3.89 2.92	4.32 2.71	4.89 2.45	5.71 2.20	
β: 2.75  	||3.37 2.96	3.70 2.78	4.11 2.56	4.67 2.31	5.48 2.07	
β: 3.0  	||3.27 2.87	3.53 2.65	3.93 2.44	4.49 2.21	5.28 1.97	


In [14]:
# So why the (slight) disagreement? I don't really know.
# It may well be that Julia has a much better control of errors, 
# certainly the optimisation defaults seem to be at the Machine-precision end of the world, 
# and a lot of intermediates will be Float64 or larger.

# The objective function seems quite flat in the vicinity of the optimum, so this would also explain some of the variance.

In [15]:
# OK, now Ref 9; Schultz PR Volume 116, Number 3, Nov 1959
const MassElectron = 9.10938188e-31;                          # kg
v=7.15 
w=6.51

m=1.0 #MassElectron
# Eqn (2.4)
μ=m*(v^2-w^2)/v^2
println("Reduced mass: $μ")
rf=(3/2*μ*v)^(1/2)
println("Feynman Polaron Radius: $rf")

# Units?!? Bohr?

Reduced mass: 0.17100885128857163
Feynman Polaron Radius: 1.3542783798281395


In [16]:
# P.S. Versions of packages used here:
Pkg.status()

17 required packages:
 - ApproxFun                     0.5.0
 - BenchmarkTools                0.0.7
 - Combinatorics                 0.3.2
 - DataArrays                    0.3.12
 - DataFrames                    0.8.5
 - GR                            0.19.0
 - HDF5                          0.7.3
 - IJulia                        1.4.1
 - Images                        0.7.0
 - Lazy                          0.11.5
 - ODE                           0.3.0
 - Optim                         0.7.4
 - Plots                         0.10.3
 - PyPlot                        2.3.1              master
 - QuadGK                        0.1.1
 - SIUnits                       0.1.0
 - UnicodePlots                  0.2.2
71 additional packages:
 - AxisArrays                    0.0.3
 - BandedMatrices                0.2.1
 - BinDeps                       0.4.5
 - Blosc                         0.1.7
 - Calculus                      0.2.0
 - CatIndices                    0.0.1
 - ColorTypes                    